In [1]:
import cudf
import cuspatial as csp

In [2]:
%%time
df = cudf.read_parquet("https://pydatatt.s3.amazonaws.com/Combined/ALL_GOM_tanker_2019-2021.parquet")[['BaseDateTime', 'MMSI', 'LAT', 'LON']].reset_index(drop=True)
df['MMSI'] = df.MMSI.astype('int32')
df['BaseDateTime'] = df.BaseDateTime.astype('datetime64[ms]')
df = df.sort_values(by=['MMSI','BaseDateTime']).reset_index(drop=True)
df
# objects, traj_offsets = csp.derive_trajectories(df.MMSI, df.LON, df.LAT, df.Time)
# objects

CPU times: user 8.94 s, sys: 2.32 s, total: 11.3 s
Wall time: 34 s


,BaseDateTime,MMSI,LAT,LON
0,2019-01-02 19:05:00,3791472,26.13211,-92.04048
1,2019-01-02 19:10:00,3791472,26.13213,-92.04049
2,2019-01-02 19:15:00,3791472,26.13215,-92.04044
3,2019-01-02 19:20:00,3791472,26.13216,-92.04049
4,2019-01-02 19:25:00,3791472,26.13212,-92.04046
...,...,...,...,...
38903728,2021-06-22 13:50:00,710033290,28.19147,-94.51869
38903729,2021-06-22 14:50:00,710033290,28.00145,-94.46387
38903730,2021-06-22 15:30:00,710033290,27.87318,-94.41796
38903731,2020-01-24 15:25:00,866666666,29.42988,-93.64103


In [3]:
mmsi_names = df.MMSI.unique().to_arrow().to_pylist()
mmsi_names[:5]

[3791472, 205042000, 205086000, 205089000, 205097000]

In [4]:
df['Unix_Seconds'] = df.BaseDateTime.astype('datetime64[s]').astype('int') - 1546315200.0 # 2019-1-1 in seconds
df

,BaseDateTime,MMSI,LAT,LON,Unix_Seconds
0,2019-01-02 19:05:00,3791472,26.13211,-92.04048,140700.0
1,2019-01-02 19:10:00,3791472,26.13213,-92.04049,141000.0
2,2019-01-02 19:15:00,3791472,26.13215,-92.04044,141300.0
3,2019-01-02 19:20:00,3791472,26.13216,-92.04049,141600.0
4,2019-01-02 19:25:00,3791472,26.13212,-92.04046,141900.0
...,...,...,...,...,...
38903728,2021-06-22 13:50:00,710033290,28.19147,-94.51869,78054600.0
38903729,2021-06-22 14:50:00,710033290,28.00145,-94.46387,78058200.0
38903730,2021-06-22 15:30:00,710033290,27.87318,-94.41796,78060600.0
38903731,2020-01-24 15:25:00,866666666,29.42988,-93.64103,33564300.0


In [6]:
%%time
list_of_rows = []
for mmsi in mmsi_names[:]:
    tmp = df[df.MMSI == mmsi][['LON','LAT','Unix_Seconds']]
    path = tmp[['LON','LAT']].values.tolist()
    ts = tmp['Unix_Seconds'].values.tolist()
    list_of_rows.append([mmsi, path, ts])
traj_df = cudf.DataFrame(list_of_rows, columns=['mmsi','path','ts'])
traj_df

CPU times: user 43.1 s, sys: 2.28 s, total: 45.4 s
Wall time: 45.3 s


,mmsi,path,ts
0,3791472,"[[-92.04048, 26.13211], [-92.04049, 26.13213],...","[140700.0, 141000.0, 141300.0, 141600.0, 14190..."
1,205042000,"[[-89.2095, 27.24959], [-89.26068, 27.33388], ...","[80076600.0, 80077800.0, 80078100.0, 80078400...."
2,205086000,"[[-93.48448, 26.39778], [-93.56166, 26.50175],...","[77814000.0, 77816700.0, 77819400.0, 77821800...."
3,205089000,"[[-90.35513, 24.80777], [-90.36815, 24.81126],...","[94534200.0, 94535100.0, 94536000.0, 94536300...."
4,205097000,"[[-93.45964, 26.5793], [-93.6274, 26.76886], [...","[82272300.0, 82275300.0, 82275600.0, 82276200...."
...,...,...,...
2972,642122021,"[[-93.82248, 25.65528], [-93.85108, 25.66127],...","[19596300.0, 19596900.0, 19597500.0, 19597800...."
2973,645337000,"[[-91.57331, 26.81737], [-91.60873, 26.85394],...","[17980500.0, 17981400.0, 17982600.0, 17982900...."
2974,710016250,"[[-88.35071, 25.61065], [-88.36058, 25.62274],...","[12231000.0, 12231300.0, 12231600.0, 12231900...."
2975,710033290,"[[-92.98647, 26.33258], [-93.03388, 26.37598],...","[76974900.0, 76976100.0, 76976400.0, 76977000...."


In [19]:
df.groupby('MMSI')[['LON','LAT']].collect()

,LON,LAT
MMSI,,
3791472,"[-92.04048, -92.04049, -92.04044, -92.04049, -...","[26.13211, 26.13213, 26.13215, 26.13216, 26.13..."
205042000,"[-89.2095, -89.26068, -89.26763, -89.28766, -8...","[27.24959, 27.33388, 27.35382, 27.36854, 27.39..."
205086000,"[-93.48448, -93.56166, -93.64222, -93.70252, -...","[26.39778, 26.50175, 26.6156, 26.71759, 26.744..."
205089000,"[-90.35513, -90.36815, -90.38229, -90.38444, -...","[24.80777, 24.81126, 24.81579, 24.81761, 24.81..."
205097000,"[-93.45964, -93.6274, -93.64242, -93.67826, -9...","[26.5793, 26.76886, 26.78581, 26.82626, 26.878..."
...,...,...
642122021,"[-93.82248, -93.85108, -93.88848, -93.91856, -...","[25.65528, 25.66127, 25.67218, 25.67529, 25.67..."
645337000,"[-91.57331, -91.60873, -91.66131, -91.67667, -...","[26.81737, 26.85394, 26.91209, 26.92953, 26.96..."
710016250,"[-88.35071, -88.36058, -88.36788, -88.37417, -...","[25.61065, 25.62274, 25.63502, 25.65427, 25.67..."
